In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [4]:
from CREDIT_UTILITY.constants import *
from CREDIT_UTILITY.utils.common import read_yaml, create_directories

In [43]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [74]:
pip install urllib

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement urllib (from versions: none)
ERROR: No matching distribution found for urllib

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [175]:
import os
import sys
from pathlib import Path
from urllib.request import Request
import zipfile
from CREDIT_UTILITY.logger import logger
from CREDIT_UTILITY.utils.common import get_size
from CREDIT_UTILITY.Exception import CustomException

In [176]:

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        print("Source URL:", self.config.source_URL)
        print("Local path",self.config.local_data_file)
        retries = 3  # Number of retry attempts
        for attempt in range(retries):
            try:
                if not os.path.exists(self.config.local_data_file):
                    filename, headers = request.urlretrieve(
                    url=self.config.source_URL,
                    filename=self.config.local_data_file
                )
                    logger.info(f"{filename} downloaded with the following info: \n{headers}")
                else:
                    logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")
            
            except Exception as e:
                 error = CustomException(e, sys)
                 logger.info(error.error_message)
       

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
  

In [185]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    error = CustomException(e, sys)
    logger.info(error.error_message)

[2024-02-24 17:25:13,766: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-02-24 17:25:13,774: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-24 17:25:13,783: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-02-24 17:25:13,783: INFO: common: created directory at: artifacts]
[2024-02-24 17:25:13,791: INFO: common: created directory at: artifacts\data_ingestion]


Source URL: https://github.com/Neeraj2210/Data_Repo/raw/main/Phishing.zip
Local path artifacts\data_ingestion\data.zip
[2024-02-24 17:25:15,309: INFO: 3639363258: artifacts\data_ingestion\data.zip downloaded with the following info: 
Connection: close
Content-Length: 240603
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "ce25cc3d9c9e3683e98a5b0dafdf768253b62488285b6af2443e211d0b71dd76"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: C19A:1951ED:200BD3:298974:65D9D921
Accept-Ranges: bytes
Date: Sat, 24 Feb 2024 11:55:14 GMT
Via: 1.1 varnish
X-Served-By: cache-bom4743-BOM
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1708775715.699911,VS0,VE260
Vary: Authorization,Accept-Encoding,Origin
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: 05244cdbadc3f5a4cf

In [157]:
%pwd

'E:\\Project\\DS Project\\E to E\\ML\\Credit Utility'